# Create Tables

This notebook provides an example for creating databases, schemas, tables, and columns using the Secoda API. To get started, you will need to [obtain an API key](https://app.secoda.co/settings/api)

## Configure `requests` with your API key

Input your API key and Secoda API endpoint URL below. The URL for the cloud instance is `https://api.secoda.co`. If you are self-hosting Secoda or on the EU instance, you will have to update variable.

In [1]:
import requests

API_KEY = ""
SECODA_API_URL = "https://api.secoda.co"
INTEGRATION_ID = ""

session = requests.Session()
session.headers.update(dict(
    Authorization=f"Bearer {API_KEY}"
))

def build_url(url: str):
    return f"{SECODA_API_URL}{url}"

In [9]:
import pandas

dataframe = pandas.read_csv("data/data_import.csv", sep=";", encoding="unicode_escape", low_memory=False)
dataframe["table"] = dataframe["name"]
dataframe["table_description"] = dataframe["description"]
dataframe["column"] = dataframe["col_name"]
dataframe.fillna('', inplace=True)
dataframe

,schema,name,description,col_name,col_type,col_description,col_sort_order,is_pk,reference_tbl,reference_col,table,table_description,column
0,orddata,orddcm_document_types,,doc_type,varchar2,,1,False,,,orddcm_document_types,,doc_type
1,orddata,orddcm_document_types,,schema_url,varchar2,,2,False,,,orddcm_document_types,,schema_url
2,orddata,orddcm_document_types,,doc_type_dsc,varchar2,,3,False,,,orddcm_document_types,,doc_type_dsc
3,orddata,orddcm_documents,,doc_id,number,,1,False,,,orddcm_documents,,doc_id
4,orddata,orddcm_documents,,doc_name,varchar2,,2,False,,,orddcm_documents,,doc_name
...,...,...,...,...,...,...,...,...,...,...,...,...,...
128929,ods_admin,ft_tr_precios,,plazo_residual,number,,18,False,,,ft_tr_precios,,plazo_residual
128930,ods_admin,ft_tr_precios,,precio_cierre,number,,19,False,,,ft_tr_precios,,precio_cierre
128931,ods_admin,ft_tr_precios,,tasa_mercado_cierre,number,,20,False,,,ft_tr_precios,,tasa_mercado_cierre
128932,ods_admin,ft_tr_precios,,tasa_emision,number,,21,False,,,ft_tr_precios,,tasa_emision


## Create a cluster and database

Secoda organizes in a Cluster -> Database -> Schema -> Table -> (Column, Query, Test) hierarchy. The first step is to create a cluster.

In [ ]:
# first make sure no clusters with the same name exist to avoid inserting duplicate entries
my_cluster_name = "oracle"
clusters = session.get(
    build_url(f"/table/clusters?title={my_cluster_name}")
).json().get("results", [])

# create a cluster if non exist with the name
if len(clusters) == 0:
    cluster = session.post(build_url("/table/clusters/"), json=dict(
        title=my_cluster_name,
        integration=INTEGRATION_ID,
    )).json()
else:
    cluster = clusters[0]


databases = session.get(
    build_url(f"/table/databases?title=bice&parent_id={cluster['id']}")
).json().get("results", [])

if len(databases) == 0:
    database = session.post(build_url("/table/databases/"), json=dict(
        title="bice",
        parent=cluster["id"],
        integration=INTEGRATION_ID,
    )).json()
else:
    database = databases[0]

cluster
database

# Process tables

In [21]:
def get_or_create_schema(schema_name: str):
    schemas = (
        session.get(
            build_url(f"/table/schemas/?title={schema_name}&parent_id={database['id']}")
        )
        .json()
        .get("results", [])
    )

    if len(schemas) == 0:
        schema = session.post(
            build_url("/table/schemas/"),
            json=dict(
                title=schema_name,
                parent=database["id"],
                integration=INTEGRATION_ID,
            ),
        ).json()
    else:
        schema = schemas[0]
    return schema


def get_or_create_table(schema_id: str, table_dict: dict):
    tables = (
        session.get(
            build_url(f"/table/tables/?title={table_dict['table']}&parent_id={schema_id}")
        )
        .json()
        .get("results", [])
    )

    if len(tables) == 0:
        table = session.post(
            build_url("/table/tables/"),
            json=dict(
                title=table_dict["table"],
                parent=schema_id,
                schema=table_dict["schema"],
                database=database["title"],
                cluster=cluster["title"],
                description=table_dict["description"],
                definition="",
                integration=INTEGRATION_ID,
            ),
        ).json()
    else:
        table = tables[0]
    return table


def get_or_create_column(schema_name: str, table_id: str, column_dict: dict):
    columns = (
        session.get(
            build_url(f"/table/columns/?title={column_dict['column']}&parent_id={table_id}")
        )
        .json()
        .get("results", [])
    )

    if len(columns) == 0:
        column = session.post(
            build_url("/table/columns/"),
            json=dict(
                title=column_dict["column"],
                parent=table_id,
                table=column_dict["table"],
                schema=column_dict["schema"],
                database=database["title"],
                cluster=cluster["title"],
                description=column_dict["col_description"],
                type=column_dict["col_type"],
                is_pk=column_dict["is_pk"],
                sort_order=column_dict["col_sort_order"],
                integration=INTEGRATION_ID,
            ),
        ).json()
    else:
        column = columns[0]


for index, entry in enumerate(dataframe.to_dict("records")):
    print(f"Processing {index}/{dataframe.shape[0]}")
    if entry["schema"] not in ["efernandez_ap", "csalazar", "ods_admin"]:
        continue
    schema = get_or_create_schema(entry["schema"])
    table = get_or_create_table(
        schema["id"],
        {
            "schema": entry["schema"],
            "table": entry["table"],
            "description": entry["description"],
        },
    )
    column = get_or_create_column(
        schema["id"],
        table["id"],
        {
            "schema": entry["schema"],
            "table": entry["table"],
            "column": entry["column"],
            "col_description": entry["col_description"],
            "col_type": entry["col_type"],
            "is_pk": entry["is_pk"],
            "col_sort_order": entry["col_sort_order"],
        },
    )


Processing 0/128934
Processing 1/128934
Processing 2/128934
Processing 3/128934
Processing 4/128934
Processing 5/128934
Processing 6/128934
Processing 7/128934
Processing 8/128934
Processing 9/128934
Processing 10/128934
Processing 11/128934
Processing 12/128934
Processing 13/128934
Processing 14/128934
Processing 15/128934
Processing 16/128934
Processing 17/128934
Processing 18/128934
Processing 19/128934
Processing 20/128934
Processing 21/128934
Processing 22/128934
Processing 23/128934
Processing 24/128934
Processing 25/128934
Processing 26/128934
Processing 27/128934
Processing 28/128934
Processing 29/128934
Processing 30/128934
Processing 31/128934
Processing 32/128934
Processing 33/128934
Processing 34/128934
Processing 35/128934
Processing 36/128934
Processing 37/128934
Processing 38/128934
Processing 39/128934
Processing 40/128934
Processing 41/128934
Processing 42/128934
Processing 43/128934
Processing 44/128934
Processing 45/128934
Processing 46/128934
Processing 47/128934
Pr

JSONDecodeError: Expecting value: line 1 column 1 (char 0)